## MLflow's Model Registry

In [3]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [9]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='file:///c:/Users/joaov/Documents/_workspace/mlops-zoomcamp-jlima/02-experiment-tracking/mlruns/2', creation_time=1685037583334, experiment_id='2', last_update_time=1685037583334, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='file:///c:/Users/joaov/Documents/_workspace/mlops-zoomcamp-jlima/02-experiment-tracking/mlruns/1', creation_time=1685023953422, experiment_id='1', last_update_time=1685023953422, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [5]:
client.create_experiment(name="my-cool-experiment")

'2'

Let's check the latest versions for the experiment with id `1`...

In [10]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [11]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 869be6e85fc04fcdbf8508eb7de9c2e6, rmse: 6.3024
run id: 1d87724aae4c4cba9b42f281763a72d4, rmse: 6.3024
run id: 41f1752e4d3942b2a640673a3a1ebc0e, rmse: 6.3024
run id: 2e26574758244dacbfa4aa75ca4c2c05, rmse: 6.3125
run id: ca960af0ef6f499a89cca7ed3d1cb74f, rmse: 6.3371


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [12]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [29]:
run_id = "869be6e85fc04fcdbf8508eb7de9c2e6"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2023/05/25 15:19:34 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 4
Created version '4' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1685038774371, current_stage='None', description=None, last_updated_timestamp=1685038774371, name='nyc-taxi-regressor', run_id='869be6e85fc04fcdbf8508eb7de9c2e6', run_link=None, source='file:///c:/Users/joaov/Documents/_workspace/mlops-zoomcamp-jlima/02-experiment-tracking/mlruns/1/869be6e85fc04fcdbf8508eb7de9c2e6/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [30]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 3, stage: Staging
version: 4, stage: None


In [39]:
model_version = 4
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1685038774371, current_stage='Production', description='The model version 4 was transitioned to Staging on 2023-05-25', last_updated_timestamp=1685038849116, name='nyc-taxi-regressor', run_id='869be6e85fc04fcdbf8508eb7de9c2e6', run_link=None, source='file:///c:/Users/joaov/Documents/_workspace/mlops-zoomcamp-jlima/02-experiment-tracking/mlruns/1/869be6e85fc04fcdbf8508eb7de9c2e6/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [32]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1685038774371, current_stage='Staging', description='The model version 4 was transitioned to Staging on 2023-05-25', last_updated_timestamp=1685038786194, name='nyc-taxi-regressor', run_id='869be6e85fc04fcdbf8508eb7de9c2e6', run_link=None, source='file:///c:/Users/joaov/Documents/_workspace/mlops-zoomcamp-jlima/02-experiment-tracking/mlruns/1/869be6e85fc04fcdbf8508eb7de9c2e6/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [23]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [24]:
! wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet -P data/

--2023-05-25 15:17:45--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:200c:5400:b:20a5:b140:21, 2600:9000:200c:2e00:b:20a5:b140:21, 2600:9000:200c:a800:b:20a5:b140:21, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:200c:5400:b:20a5:b140:21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1474538 (1,4M) [binary/octet-stream]
Saving to: 'data/green_tripdata_2021-03.parquet'

     0K .......... .......... .......... .......... ..........  3%  348K 4s
    50K .......... .......... .......... .......... ..........  6%  477K 3s
   100K .......... .......... .......... .......... .......... 10%  762K 3s
   150K .......... .......... .......... .......... .......... 13%  889K 2s
   200K .......... .......... .......... .......... .......... 17% 1,32M 2s
   250K .......... .......... .......... .......... .....

In [26]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [33]:
print(run_id)

869be6e85fc04fcdbf8508eb7de9c2e6


In [34]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

C:\Users\joaov\AppData\Local\Temp\ipykernel_2964\2852634549.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')


'c:\\Users\\joaov\\Documents\\_workspace\\mlops-zoomcamp-jlima\\02-experiment-tracking\\preprocessor'

In [35]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [36]:
X_test = preprocess(df, dv)

In [37]:
target = "duration"
y_test = df[target].values

In [40]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

2023/05/25 15:21:00 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


[15:21:00] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: total: 39.6 s
Wall time: 7.86 s


{'rmse': 6.248294328516825}

In [41]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

2023/05/25 15:21:48 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


[15:21:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: total: 39 s
Wall time: 6.76 s


{'rmse': 6.248294328516825}

In [43]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: creation_timestamp=1652971637398, current_stage='Production', description='The model version 4 was transitioned to Staging on 2022-05-19', last_updated_timestamp=1652972763255, name='nyc-taxi-regressor', run_id='b8904012c84343b5bf8ee72aa8f0f402', run_link=None, source='./mlruns/1/b8904012c84343b5bf8ee72aa8f0f402/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>